In [1]:
#Extract
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *

# Set up environment
cc.setupEnvironment()
cc.listEnvironment()

# Start local Spark cluster
spark = cc.startLocalCluster("vehicle_dim")
spark.getActiveSession()  # To get the active Spark session

# Extract data from the "vehicles" table
cc.set_connectionProfile("veloDB")

# EXTRACT: Loading data from the vehicles table into df_operational_vehicle_dim
df_operational_vehicle_dim = (spark.read
    .format("jdbc")
    .option("driver", cc.get_Property("driver")) 
    .option("url", cc.create_jdbc()) 
    .option("dbtable", "bike_types")
    .option("user", cc.get_Property("username")) 
    .option("password", cc.get_Property("password"))
    .load())

# Show the first 20 rows of the operational data to confirm extraction
df_operational_vehicle_dim.show(20)


ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 1980
JPY_INTERRUPT_EVENT: 1980
JPY_PARENT_PID: 1812
JPY_SESSION_NAME: DWH_VehicleDim.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAP

In [2]:
#Load

df_operational_vehicle_dim.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vehicle_dim")


# Confirmation that the data has been saved to the Delta table
print("Data successfully written to Delta table: vehicle_dim")

Data successfully written to Delta table: vehicle_dim


In [4]:
#Test
# Query the saved Delta table
df_operational_vehicle_dim.show()


+----------+-------------------+
|biketypeid|biketypedescription|
+----------+-------------------+
|         1|          Velo Bike|
|         2|        Velo E-Bike|
|         3|               Step|
|         4|            Scooter|
+----------+-------------------+



In [5]:
spark.sql("DESCRIBE FORMATTED vehicle_dim").show(truncate=False)


+----------------------------+------------------------------------------------------------+-------+
|col_name                    |data_type                                                   |comment|
+----------------------------+------------------------------------------------------------+-------+
|biketypeid                  |int                                                         |NULL   |
|biketypedescription         |string                                                      |NULL   |
|                            |                                                            |       |
|# Detailed Table Information|                                                            |       |
|Name                        |spark_catalog.default.vehicle_dim                           |       |
|Type                        |MANAGED                                                     |       |
|Location                    |file:/C:/KDG/dataai_groupproject/spark-warehouse/vehicle_dim|       |


In [7]:
# Create a temporary view from the Delta table
spark.sql("CREATE OR REPLACE TEMP VIEW vehicle_dim_view AS SELECT * FROM vehicle_dim")

vehicle_dim_view = spark.sql("SELECT * FROM vehicle_dim_view")
vehicle_dim_view.show()


+----------+-------------------+
|biketypeid|biketypedescription|
+----------+-------------------+
|         1|          Velo Bike|
|         2|        Velo E-Bike|
|         3|               Step|
|         4|            Scooter|
+----------+-------------------+

